In [1]:
import os
os.chdir("../")

In [2]:
# access the dagshub repository
import dagshub
dagshub.init(repo_owner='zeineb.udacity', repo_name='my-first-repo', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

C:\Users\user\AppData\Roaming\Python\Python312\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" 
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=8e8903ce-0d0f-41dc-b200-acb8a35a7e9c&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=18f012debb704865601b89e6157ccaa02b7df2c031bb0de103c260e11d29e6c2




Accessing as zeineb.udacity

Initialized MLflow to track repo "zeineb.udacity/my-first-repo"

Repository zeineb.udacity/my-first-repo initialized!

🏃 View run ambitious-bee-478 at: https://dagshub.com/zeineb.udacity/my-first-repo.mlflow/#/experiments/0/runs/aea495f51f004a02a6531ad7aa176992
🧪 View experiment at: https://dagshub.com/zeineb.udacity/my-first-repo.mlflow/#/experiments/0


In [ ]:
import tensorflow as tf
model = tf.keras.models.loadf_model("artifacts/training/model.h5")


In [3]:
# update the entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [5]:
from cnnDiseaseClassifier.constants import *
from cnnDiseaseClassifier.utils.common import read_yaml, create_directories, save_json

In [7]:
# configuration manager
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/zeineb.udacity/my-first-repo.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [6]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [9]:
# update the component
class Evaluation:
    def __init__(self,config:EvaluationConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
             # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

    

In [ ]:
# pipeline
try:
    config=ConfigurationManager()
    model_evaluation_config = config.get_evaluation_config()
    model_evaluation = Evaluation(config= model_evaluation_config)
    model_evaluation._valid_generator()
    model_evaluation.evaluation()
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e


